# Clasificación por medio del método K-NN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_csv('Data/aps_failure_training_set.csv',na_values='na')
df_test = pd.read_csv('Data/aps_failure_test_set.csv', na_values='na')

In [3]:
feature_columns = df_train.drop(columns=['class']).columns.values
features = np.empty((1,1))

Se eliminarán aquellas columnas que posean menos de 50000 datos (número total de datos de entrenamiento : 60000).

In [4]:
for feature in feature_columns:
    if len(df_train.loc[df_train[feature].notnull()]) > 50000:
        features = np.append(features,feature)
        
features = np.delete(features,0,0)

Se completarán los datos faltantes con la mediana.

In [5]:
df_train[features]=df_train[features].fillna(df_train.median())
df_test[features]=df_test[features].fillna(df_test.median())

Se procederá a reemplazar la clase {neg, pos} por {-1, 1} respectivamente.

In [6]:
df_train['target'] = df_train['class'].replace ({'neg':-1, 'pos':1})
df_test['target'] = df_test['class'].replace ({'neg':-1, 'pos':1})
print( 'df_train target:'), print(df_train['target'].value_counts())
print( 'df_test target:'), print(df_test['target'].value_counts())

df_train target:
-1    59000
 1     1000
Name: target, dtype: int64
df_test target:
-1    15625
 1      375
Name: target, dtype: int64


(None, None)

Se separará un grupo de validación dentro del conjunto de entrenamiento. Se utilizirá un valor de random_state = 1 para mantener uniformidad en las diferentes pruebas que se realicen.

In [7]:
from sklearn.model_selection import train_test_split

X = df_train[features]
y = df_train['target']

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1, random_state = 1)
print('X_train:', X_train.shape); print('y_train:', y_train.shape)
print('X_val:', X_val.shape); print('y_val:', y_val.shape)

X_train: (54000, 145)
y_train: (54000,)
X_val: (6000, 145)
y_val: (6000,)


In [8]:
def projectData(pcaModel, X, K):
    Z = pcaModel.transform(X)[:,:K]
    return Z

In [9]:
def recoverData(pcaModel, Z):
    K = Z.shape[1]
    X_rec = np.dot(Z, pcaModel.components_[:K,:])
    return X_rec

Se procederá a realizar una reducción de dimensionalidad usando el método PCA. Se escogieron los primeros 60 componentes principales.

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler().fit(X)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)


pca = PCA().fit(X_train_scaled)
pca_loadings = pca.components_
pca_scores_train = projectData(pca,X_train_scaled,60)
pca_scores_val = projectData(pca,X_val_scaled,60)
X_train_pca = recoverData(pca,pca_scores_train)
X_val_pca = recoverData(pca,pca_scores_val)

In [11]:
X_train_pca.shape

(54000, 145)

Se entrena un modelo K-NN con los datos con dimensionalidad reducida.

In [12]:
from sklearn.neighbors import KNeighborsClassifier
modelo = KNeighborsClassifier(n_neighbors = 60)
modelo.fit(X_train_pca, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=60, p=2,
           weights='uniform')

Se define la función de costo a utilizar para mejorar el modelo. Se decidió variar el umbral de clasificación con el objetivo de disminuir la cantidad de falsos negativos.

In [13]:
def Costo_total(probabilidades, y, umbral):
    U_check = 10 #cost that an unnecessary check
    M_check = 500 # cost of missing a faulty truck

    FN = ((probabilidades < umbral) & (y == 1)).sum()
    TN = ((probabilidades < umbral) & (y == -1)).sum()
    FP = ((probabilidades >= umbral) & (y == -1)).sum()
    TP = ((probabilidades >= umbral) & (y == 1)).sum()

    Costo_total = FP*U_check + FN*M_check
    return Costo_total

In [14]:
prob = modelo.predict_proba(X_val_pca)[:,1]

Buscando el mejor umbral para minimizar el costo.

In [15]:
valores = np.arange(0, 1, 0.05)
mejor_costo = 10000
mejor_umbral = 0
for i in valores:
    Total_Cost = Costo_total(prob, y_val, i)
    if Total_Cost < mejor_costo:
        mejor_costo = Total_Cost
        mejor_umbral = i

print('Total Cost: ', mejor_costo, mejor_umbral)

Total Cost:  6390 0.05


Se obtiene la función de costo en el conjunto de prueba.

In [16]:
X_test = df_test[features]
y_test = df_test['target']
X_test_scaled = scaler.transform(X_test)
pca_scores_test = projectData(pca,X_test_scaled,60)
X_test_pca = recoverData(pca,pca_scores_test)

probabilidades_test = modelo.predict_proba(X_test_pca)[:,1]
Costo_total(probabilidades_test,y_test, mejor_umbral)

19420